## Run Testset and Just look at the results

In [ ]:
from dotenv import load_dotenv
import os
import gspread
import pandas as pd
import requests
from typing import List, Optional
from askem.preprocessing import get_all_cap_words

load_dotenv()

In [ ]:
def load_testset() -> pd.DataFrame:
    """Load testset from Google Sheet."""

    GCP_SECRET_FILE_PATH = os.getenv("GCP_SECRET_FILE_PATH")

    gc = gspread.service_account(filename=GCP_SECRET_FILE_PATH)
    sheet = gc.open("ASKEM-TA1-testset").worksheet("questions")

    records = sheet.get_values()
    labels = records[0]
    data = records[1:]

    new_labels = [label.lower().replace(" ", "_") for label in labels]
    df = pd.DataFrame.from_records(data, columns=new_labels)
    return df[["source", "target_type", "is_keyword", "question"]]

In [ ]:
df["terms"] = df["question"].apply(get_all_cap_words)

In [ ]:
df[["terms", "is_keyword"]]

It seems that we should not include some really common terms like: `COVID`

In [ ]:
BLACKLIST_COMMON_TERMS = ["COVID19", "COVID"]


def get_better_terms(text: str, blacklist: Optional[List[str]] = None) -> List[str]:
    """Get better terms from text."""
    terms = get_all_cap_words(text)
    if not terms:
        return None

    if blacklist is None:
        blacklist = BLACKLIST_COMMON_TERMS

    better_terms = [term for term in terms if term not in blacklist]
    if not better_terms:
        return None
    return better_terms

In [ ]:
df["terms"] = df["question"].apply(get_better_terms)
df[["terms", "is_keyword", "question"]]

Note. `COmplexVID` is not considered as a term. TODO: May need to address this later.

In [ ]:
def eval_ta1(
    question: str, article_terms: List[str] = None, paragraph_terms: List[str] = None
) -> List[dict]:
    """Evaluate a question using the retriever API."""

    URL = os.getenv("RETRIEVER_URL")
    APIKEY = os.getenv("RETRIEVER_APIKEY")

    headers = {"Content-Type": "application/json", "Api-Key": APIKEY}
    json = {"question": question, "top_k": 10, "doc_type": "paragraph"}

    if article_terms:
        json["article_terms"] = article_terms

    if paragraph_terms:
        json["paragraph_terms"] = paragraph_terms

    response = requests.post(URL, headers=headers, json=json)
    return response.json()

In [ ]:
# Get all questions with terms

df_with_terms = df.query("terms.notnull()")

results = []
for row in df_with_terms.itertuples():
    results.append(
        {
            "question": row.question,
            "terms": row.terms,
            "results_with_article_level_filter": eval_ta1(
                row.question, article_terms=row.terms
            ),
            "results_with_paragraph_level_filter": eval_ta1(
                row.question, paragraph_terms=row.terms
            ),
        }
    )

In [ ]:
results